[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weatherpal-ai/mcp_handle/blob/main/examples/crewai/Use_MCPHandle_with_CrewAI.ipynb)


# CrewAI + MCP Handle Integration

This tutorial demonstrates how to use CrewAI with MCPHandle MCP (Model Context Protocol) servers to create powerful AI agent crews with access to external tools and services.


## Prerequisites

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **MCPHandle API key** - Get at [mcp-handle.dev](https://www.mcp-handle.dev/)

In [2]:
# Install the required packages
%pip install -qU crewai 'crewai-tools[mcp]' mcp_handle openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import MCPServerAdapter
from mcp_handle import MCPHandle
from mcp_handle.types import McpServerName

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["MCP_HANDLE_API_KEY"] = "YOUR_MCP_HANDLE_API_KEY"  # Replace with your actual MCPHandle API key


## Case Study 1: YouTube Research Crew

Let's create a CrewAI crew that can research YouTube videos using MCPHandle MCP servers.


#### Step 1 - Create YouTube MCP Server using MCPHandle


In [5]:
mcp_handle_client = MCPHandle(api_key=os.getenv("MCP_HANDLE_API_KEY"))

youtube_mcp_instance = mcp_handle_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
)

print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}")
print(f"📝 Instance ID: {youtube_mcp_instance.instance_id}")


🔗 YouTube MCP server created at: https://youtube-mcp-server.mcp-handle.dev/mcp/?instance_id=270cbd51-e737-407d-85ce-6e6162248671
📝 Instance ID: 270cbd51-e737-407d-85ce-6e6162248671


#### Step 2 - Create CrewAI Agents with MCP Tools


In [7]:
# Configure MCP server parameters for CrewAI
server_params = {
    "url": youtube_mcp_instance.server_url,
    "transport": "streamable-http"
}

YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick the video you want to analyze

try:
    with MCPServerAdapter(server_params) as mcp_tools:
        print(f"✅ Available tools from YouTube MCP server: {[tool.name for tool in mcp_tools]}")
        
        # Create a YouTube Analysis Agent
        youtube_agent = Agent(
            role="YouTube Content Analyst",
            goal="Research and analyze YouTube videos to extract comprehensive insights and create structured summaries with timestamps",
            backstory="You are an expert at analyzing video content, extracting transcripts with precise timestamps, and creating professional summaries with key insights, takeaways, and time-coded references for easy navigation.",
            tools=mcp_tools,
            reasoning=True,
            verbose=False
        )
        
        # Define Task
        analysis_task = Task(
            description=f"Research the YouTube video at {YOUTUBE_VIDEO_URL}. Extract the video transcript, analyze the content, and create a comprehensive summary with key points, timestamps, and main takeaways. Structure it in a professional format.",
            expected_output="Complete video analysis with transcript, structured summary, key insights, timestamps, and main takeaways",
            agent=youtube_agent,
            markdown=True
        )
        
        # Create and execute the crew
        youtube_crew = Crew(
            agents=[youtube_agent],
            tasks=[analysis_task],
            verbose=False, # set to True to print the reasoning process
            process=Process.sequential
        )
        
        result = youtube_crew.kickoff()
        print(result)
        
except Exception as e:
    print(f"❌ Error connecting to YouTube MCP server: {e}")
    print("Ensure the MCP server is running and accessible.")


/Users/zihaolin/src/mcp_handle/.venv/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'items', 'anyOf', 'enum', 'properties'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


✅ Available tools from YouTube MCP server: ['get_youtube_video_transcript']


/Users/zihaolin/src/mcp_handle/.venv/lib/python3.13/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

```
# YouTube Video Analysis: Andrej Karpathy - Software Is Changing (Again)

## Video Details
- **Title:** Andrej Karpathy: Software Is Changing (Again)
- **Channel:** Y Combinator
- **Published On:** June 19, 2025
- **Duration:** 39:32
- **View Count:** 1,035,413
- **Like Count:** 33,783
- **Comment Count:** 750
- **Thumbnail:** ![Thumbnail](https://i.ytimg.com/vi/LCEmiRjPEtQ/hqdefault.jpg)

## Key Insights Summary

### Introduction
- **[00:00 - Intro]**  
  Andrej Karpathy introduces the topic of the keynote, discussing the evolution of software and the significance of
new technological paradigms.

### Software Evolution: From 1.0 to 3.0
- **[01:25 - Software evolution: From 1.0 to 3.0]**  
  Karpathy explains the major shifts in software development, highlighting that we are transitioning into "Software
3.0," where natural language processing (NLP) takes center stage.

### Rise of Software 3.0
- **[04:40 - Programming in English: Rise of Software 3.0]**  
  In this segment, he emphasizes how the use of language models (LLMs) allows programmers to interact with software
using English, democratizing programming.

### LLMs as Utilities
- **[06:10 - LLMs as utilities, fabs, and operating systems]**  
  Karpathy compares LLMs to utilities and operating systems, proposing that they will be foundational to future 
software infrastructure.

### New LLM OS
- **[11:04 - The new LLM OS and historical computing analogies]**  
  He discusses how current developments resemble the 1960s in computing history, pointing out the potential for new
forms of computing to emerge.

### Psychology of LLMs
- **[14:39 - Psychology of LLMs: People spirits and cognitive quirks]**  
  The discussion shifts to the psychological aspects of LLMs, referring to them as "people spirits" that simulate 
human-like capabilities while retaining unique flaws.

### Designing LLM Apps
- **[18:22 - Designing LLM apps with partial autonomy]**  
  Karpathy suggests that LLM technology opens avenues for developing applications with varying degrees of autonomy.

### Human-AI Collaboration
- **[23:40 - The importance of human-AI collaboration loops]**  
  Emphasizing the need for effective collaboration between humans and AI applications, he outlines strategies for 
fostering productive interactions.

### Lessons from Tesla Autopilot
- **[26:00 - Lessons from Tesla Autopilot & autonomy sliders]**  
  He shares lessons learned from Tesla’s Autopilot system, detailing the implications for software development and 
user experience.

### The Iron Man Analogy
- **[27:52 - The Iron Man analogy: Augmentation vs. agents]**  
  The comparison of LLMs to Iron Man's suit illustrates the concept of augmentation, emphasizing the potential for 
enhanced human capabilities.

### Vibe Coding
- **[29:06 - Vibe Coding: Everyone is now a programmer]**  
  Karpathy explains 'vibe coding,' a concept where increased accessibility to programming via LLMs allows more 
people to engage in software development.

### Future-Ready Digital Infrastructure
- **[33:39 - Building for agents: Future-ready digital infrastructure]**  
  He discusses the need to build a resilient and adaptable digital infrastructure that supports LLM-driven 
applications and agents.

### Conclusion
- **[38:14 - Summary: We’re in the 1960s of LLMs — time to build]**  
  In conclusion, he reflects on the current state of LLMs, advocating for innovation and building a new kind of 
computer that embraces the possibilities of software 3.0.

## Main Takeaways
- The transition to Software 3.0 marks a significant shift where natural language processing changes how we 
interact with technology.
- LLMs have the potential to democratize programming, making it accessible to a broader audience.
- Collaborative human-AI systems will define the future of software development, with a focus on adapting to new 
paradigms.
- Historical computing analogies indicate that we are in the early stages of a transformative era, comparable to 
the 196

/Users/zihaolin/src/mcp_handle/.venv/lib/python3.13/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

## Case Study 2: Multi-Service Research Crew with Gmail Integration

Let's create a more complex crew that uses multiple MCP servers - YouTube for research and Gmail for communication.


#### Step 1 - Create Gmail MCP Server (OAuth required)


In [8]:
import webbrowser

gmail_mcp_instance = mcp_handle_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
)

webbrowser.open(gmail_mcp_instance.oauth_url)

print(f"🔐 Opening OAuth authorization for Gmail...")
print(f"📱 If not redirected automatically, open: {gmail_mcp_instance.oauth_url}")
# print(f"🔗 Gmail MCP server will be available at: {gmail_mcp_instance.server_url}")


🔐 Opening OAuth authorization for Gmail...

#### Step 2 - Create Multi-Service Crew with Multiple MCP Servers


In [13]:
VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick the video you want to analyze
RECIPIENT_EMAIL = "zihaolin@mcp-handle.dev"  # Replace with your email


# Configure multiple MCP servers
multiple_server_params = [
    {
        "url": youtube_mcp_instance.server_url,
        "transport": "streamable-http"
    },
    {
        "url": gmail_mcp_instance.server_url,
        "transport": "streamable-http"
    }
]

try:
    with MCPServerAdapter(multiple_server_params) as all_mcp_tools:
        print(f"✅ Available tools from all MCP servers: {[tool.name for tool in all_mcp_tools]}")
        
        # Create YouTube Research Agent
        youtube_research_agent = Agent(
            role="YouTube Content Analyst",
            goal="Research and analyze YouTube videos to extract comprehensive insights and create structured summaries with timestamps",
            backstory="You are an expert at analyzing video content, extracting transcripts with precise timestamps, and creating professional summaries with key insights, takeaways, and time-coded references for easy navigation.",
            tools=mcp_tools,
            reasoning=False,
            verbose=False,
        )
        
        # Create Email Communication Agent
        email_agent = Agent(
            role="Email Communications Specialist",
            goal="Draft and send professional email communications based on research findings",
            backstory="You are skilled at crafting professional emails and managing correspondence with clear, impactful messaging.",
            tools=mcp_tools,
            reasoning=True,
            verbose=False,
        )
        
        # Define workflow tasks
        youtube_research_task = Task(
            description=f"Research the YouTube video at {VIDEO_URL}. Extract transcript, analyze the content for key insights about AI and software development, and create a comprehensive analysis report with key takeaways and recommendations.",
            expected_output="Complete video analysis report with transcript, key insights, recommendations, and strategic takeaways",
            agent=youtube_research_agent
        )
        
        send_email_task = Task(
            description=f"Based on the youtube analyze, draft and send a professional email to {RECIPIENT_EMAIL} with the subject 'Youtube video AI analysis'. Include content of the youtube video in the email.",
            expected_output="Confirmation that a professional email has been sent with the research insights",
            agent=email_agent,
            context=[youtube_research_task]
        )
        
        # Create and execute the crew
        multi_service_crew = Crew(
            agents=[youtube_research_agent, email_agent],
            tasks=[youtube_research_task, send_email_task],
            verbose=False, # set to True to print the reasoning process
            process=Process.sequential
        )
        
        result = multi_service_crew.kickoff()
        print(result)
        
except Exception as e:
    print(f"❌ Error with multi-service MCP integration: {e}")
    print("Ensure all MCP servers are running and properly authenticated.")


## Security Best Practices

When using CrewAI with MCPHandle MCP servers, follow these security guidelines:


In [ ]:
# Example of secure MCP server configuration
def create_secure_mcp_crew():
    """
    Demonstrates secure MCP server integration with CrewAI
    """
    # Use environment variables for sensitive data
    api_key = os.getenv("MCP_HANDLE_API_KEY")
    if not api_key:
        raise ValueError("MCP_HANDLE_API_KEY environment variable is required")
    
    # Validate server URLs (use HTTPS in production)
    server_params = {
        "url": youtube_mcp_instance.server_url,
        "transport": "streamable-http"
    }
    
    # Always use context managers for proper resource cleanup
    try:
        with MCPServerAdapter(server_params) as mcp_tools:
            # Validate available tools before use
            if not mcp_tools:
                raise ValueError("No tools available from MCP server")
            
            print(f"✅ Securely connected with {len(mcp_tools)} tools")
            return mcp_tools
            
    except Exception as e:
        print(f"🔒 Security check failed: {e}")
        return None

# Example usage
secure_tools = create_secure_mcp_crew()
if secure_tools:
    print("✅ Secure connection established successfully")
else:
    print("❌ Secure connection failed")


## Summary

This tutorial demonstrated how to integrate CrewAI with MCPHandle MCP servers. We covered several key patterns:

### 🎯 **Key Features Demonstrated:**

1. **Single Agent YouTube Analysis**: Using 1 agent with YouTube MCP server for comprehensive video analysis
2. **Two-Agent Multi-Service Workflow**: YouTube research agent + Email communication agent using multiple MCP servers
3. **Security Best Practices**: Secure configuration and error handling

### 🚀 **CrewAI + MCPHandle Benefits:**

- **Seamless Integration**: MCPServerAdapter makes it easy to connect to MCPHandle MCP servers
- **Agent Specialization**: Different agents can have access to different tools
- **Efficient Workflows**: Streamlined crews with focused responsibilities
- **Scalable Architecture**: Easy to add more MCP servers and tools
- **Professional AI Crews**: Create sophisticated AI teams with real-world capabilities

### 🛠 **Available MCP Servers:**
MCPHandle provides MCP servers for YouTube, Gmail, Google Drive, Slack, GitHub, and many more services - all easily integrable with CrewAI!

**Happy crew building!** 🚀👥
